<a href="https://colab.research.google.com/github/Cseudave/automatic_tops/blob/main/06_Encontrar_mejor_recort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crear los videos cortos

MoviePy es una excelente librería que permite trabajar con videos usando python. Aunque no está optimizada para utilizar tarjetas gráficas, será muy útil.  

## Instalar librerías necesarias

In [ ]:
# Instalamos librerías
!pip install google-api-python-client
!pip install pytube

!pip install moviepy
!pip install imageio-ffmpeg 
!pip install imageio
!apt install imagemagick

!pip install IPython
!pip install mpld3 

!pip install fuzzywuzzy

!pip install playsound

In [ ]:
# Actulizamos librerías
!pip install --upgrade imageio-ffmpeg
!pip install --upgrade moviepy 
!pip install --upgrade imageio
!pip install --upgrade ffmpeg

In [ ]:
# Configuración adicional para trabajar con texto en movipy usando ImageMagick
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

In [ ]:
# Importamos librerías
from apiclient.discovery import build
from pytube import YouTube
import os 
import time
from bs4 import BeautifulSoup

import re
import numpy as np
from numpy import arange
import pandas as pd
import matplotlib.pyplot as plt

import csv

from IPython.core.display import HTML
import mpld3 
from scipy.stats import moment
from moviepy.editor import *

import os 

from moviepy.audio.AudioClip import concatenate_audioclips
from moviepy.editor import concatenate_videoclips
from moviepy.editor import VideoFileClip
from moviepy.editor import ColorClip
from moviepy.editor import CompositeVideoClip
from moviepy.editor import *

from IPython.display import Audio
from datetime import datetime

## Recortar mejor momento

In [ ]:
# Obtenemos un array de los promedios de intensidad por segundo del opening
def op_plot(opening):
  clip = VideoFileClip(str(opening)+'.mp4')
  audio_array = clip.audio.to_soundarray()
  data =  abs(audio_array)
  window_size = clip.audio.fps
  indices = np.arange(len(data))
  # Crea un array vacío para almacenar los promedios parciales
  averages = []

  # Itera sobre el array original con una ventana de tamaño window_size
  for i in range(0, round(len(data)/window_size) + 1):
      window = data[i*window_size:(i + 1)*window_size]
      average = np.mean(window)
      averages.append(average)
  averages = np.array(averages)
  averages = averages[~np.isnan(averages)]
  indices = np.arange(len(averages))

  # Convierte el array de promedios en un array de NumPy
  averages = np.array(averages)
  averages = averages/np.max(averages)

  # Grafica el valor del array de promedios versus su índice
  # Para poder analizarlo
  plt.plot(indices, averages)
  fig = plt.gcf()
  html = mpld3.display(fig)
  # Muestra la gráfica
  display(html)

  plt.show()
  return averages, clip.duration

In [ ]:
# Normalizamos el array de intensidades
def op_level(averages):
  averages = np.array(averages)
  x = averages/np.max(averages)
  condiciones = [x < 0.1, x >= 0.1]
  valores = [0, 1]
  resultado = np.piecewise(x, condiciones, valores) 
  # Grafica el valor del array de promedios versus su índice
  indices = np.arange(len(averages))
  
  # plt.plot(indices, resultado)
  # fig = plt.gcf()
  # html = mpld3.display(fig)
  # # Muestra la gráfica
  # display(html)

  #plt.show()
  return resultado

In [ ]:
def mask_silend(mapa, levels):
# Una funcion para discriminar las partes sin sonido

  level = np.array(
      [levels[round(n*(len(levels)/len(mapa)))] 
       for n in range(0, len(mapa) - 1)])
  
  # fig, ax = plt.subplots()
  # fig.set_size_inches(10, 2)
  data = mapa[:-1] * level

  # x = np.arange(len(data))
  # plt.plot(x, data)
  # plt.show
  return data

In [ ]:
# Buscamos el intervalo más visto y con más intensidad
def max_sample(arr, duration, n=20):
  # size es el tamaño de la muestra
  size = round(n*305/duration)
  max_sum = 0
  max_moment = np.amax(arr)
  max_index = None
  for i in range(len(arr) + 1):
    sample = arr[i : i + size]
    sample_sum = sum(sample)
    if sample_sum > max_sum and max_moment in list(sample):
      max_sum = sample_sum
      max_sample = sample
      max_index = i
  return (max_index/305)*duration

In [ ]:
# Elegimos el mejor corte según dos criterios 
# 1. El mapa de momentos más reproducidos
# 2. Si no es posible, según un comportamiento especifico buscado
def best_cut(anime, origin=-1, n=20):
  averages, duration = op_plot('ops_full/' + str(anime))
  #print(averages)
  levels = op_level(averages)
  #mapa = 0
  #print(levels)
  try:
    df = pd.read_csv('ops_maps/map_'+ str(anime) + '.csv')
    arr = mask_silend(df['0'], levels)
    #print(arr)
    a = max_sample(arr, duration, n)
    #mapa = 1
  except:
    a = 35
  if origin >= 0:
    a = origin
  b = a + n
  # print(f'rango {a}, {b}')
  dif = diferences(np.array(averages))
  a, b = choose_pair(dif, a, b, averages, 8, n)
  return a , b

In [ ]:
# Busca el intervalo con un comportamineto especifico
# Donde existe mayor contraste y se mantiene una gran intensidad
def choose_pair(arr, a, b, ave, s=8, n=20):
  a = round(a)
  b = round(b)
  max_sum = 0
  index = a
  sign = 1
  if a == 0:
    sign = 0

  pairs = [(a + i, a + s + i) for i in range(2*n)]

  for i, pair in enumerate(pairs):

    suma = (2*arr[pair[0]]  - arr[pair[0] - 1]) * ave[pair[0]] 
    if suma > max_sum:
      max_sum = suma
      index = a + i

  return index + 1, index + s + 1

In [ ]:
# Obtener los cambios de intensidad en cada segundo
def diferences(arr):
  arr1 = arr
  arr2 = arr
  arr2 = np.delete(arr2, 0)
  arr2 = np.insert(arr2, -1, 0)
  dif = arr2 - arr1
  return dif